In [ ]:
"""""
Created and Updated on Mon Aug 16th 2021

@author: Rakesh Choudhary
"""""

# Introduction

This notebook documents the WSI feature extraction process.

Briefly the steps are:

**WSI and Tissue Processing**

1. Connecting to HistomicsTK
2. Perform the Tissue Detection and identification of unique tissue areas

**Obtain Geometry Files**

3. Extract geometry, merge the polygons and save the geometry file required for feature extraction, if no tissue separation required directly jump to step 7 and eliminate the triangles crossing the MT boundary
4. Perform the cutting of triangles if there are multiple tissue slices on the WSI
5. Apply Network Graphx to find the individual objects located in the two tissue slices
6. Use the object list obtained from the above step to apply heuristics and obtain Main Tumor and Satellite regions for the respective tissue slices

**Architectural Feature Extraction**

7. Using the geom file, extract the initial set of Delaunay Features
8. Calculate the Delaunay and Satellite Distance features for the separated Tissue Slices and ask Dr. Doyle if we require convex hull features
9. Save the feature values in a Pandas data frame
10. Upload these features back on HistomicsTk for visualization


## All Imports Here

In [ ]:
import os
import sys
sys.path.insert(0, '../src/utils')
#from pathlib import Path 
import glob

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.tri as T
from matplotlib.path import Path
import matplotlib.pyplot as plt
from pylab import *

from skimage import data
from skimage.color import rgb2hsv

from PIL import Image, ImageDraw

import girder_client

import networkx as nx

from skimage import segmentation, color
from skimage.morphology import erosion, disk, opening, remove_small_objects
from skimage.measure import label, regionprops, regionprops_table

from scipy import ndimage as ndi
from scipy import stats
from scipy.spatial import distance, ConvexHull, Delaunay

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon

from dotenv import load_dotenv

from histomicstk.saliency.tissue_detection import (
    get_slide_thumbnail, get_tissue_mask)

from histomicstk.annotations_and_masks.masks_to_annotations_handler import (
    get_annotation_documents_from_contours
)

%matplotlib inline

In [ ]:
from htk_utils import (
    get_histomics_connection, 
    get_sample_id,
    get_annotations_from_htk
)

from geom_utils import (
    get_polygon_from_cords,
    get_edge_cordinates,
    get_centroid_cordinates,
    cut_triangles
)

from feature_utils import (
    get_triangle_lengths,
    get_triangle_areas,
    descriptive_stats,
    assign_wave_index
)

In [ ]:
from sunycell import dsa

# 1. Connect to Histomics / DSA

In [ ]:
# Load secrets
load_dotenv(dotenv_path='api.env')
APIURL = os.getenv('APIURL')
APIKEY = os.getenv('APIKEY')

# Get connection
conn = get_histomics_connection(APIURL, APIKEY)

In [ ]:
def get_collection_id(coll,conn):
    coll_list=conn.get("collection?text="+coll+"limit=50&sort=name&sortdir=1")
    coll_id=None
    for folder in coll_list:
        coll_id = folder['_id'] 
    return coll_id

In [ ]:
coll_list = get_collection_id("Oral Cavity Cancer", conn)
coll_list

In [ ]:
def get_folder_id(folder_name,conn,collection_id):
    folder_id = None
    folder_list = conn.listFolder(collection_id, parentFolderType = "collection")
    for folder in folder_list:
        if folder['name'] == folder_name:
            folder_id = folder['_id']
    return folder_id

In [ ]:
def get_sample_id(sample_name,conn,collection_name,folder_name):
    collection_id = get_collection_id(collection_name,conn)
    folder_id = get_folder_id(folder_name,conn,collecion_id)
    sample_id = None
    item_list = conn.listItem(folder_id)
    for item in item_list:
        if item['name']==sample_name:
            sample_id=item['_id']
    return sample_id

In [ ]:
collection_id = get_collection_id('Oral Cavity Cancer', conn)
folder_id = get_folder_id('NMHS-Finished', conn, collection_id)

In [ ]:
# Get a list of slides
item_id_list = conn.listItem(folder_id)

slide_ids  = []
slide_names = []
for l in item_id_list:
    slide_ids.append(l['_id'])
    slide_names.append(l['name'])

In [ ]:
# Make holder directory for features
feature_dir = 'features_csv'
os.makedirs(feature_dir, exist_ok=True)

# Single-slide test

In [ ]:
#Top 4:
# OCC-02-0004-01Z-03-O01.tiff_0 = 27
# OCC-05-0009-01Z-01-O01.tiff_0 = 132
# OCC-03-0021-01Z-01-O01.tiff_0 = 66
# OCC-03-0005-01Z-02-O01.tiff_0 = 53

In [ ]:
# Bottom 3:
# OCC-02-0005-01Z-01-O01.tiff_0 = 29
# OCC-02-0018-01Z-02-O01.tiff_0 = 45
# OCC-03-0035-01Z-02-O01.tiff_0 = 80

In [ ]:
slide_id = slide_ids[18]
slide_name = slide_names[18]


In [ ]:
slide_name

In [ ]:
# Grab a thumbnail of the image
try:
    thumbnail_rgb = get_slide_thumbnail(conn, slide_id)
except:
    print(f'Could not extract thumbnail for {slide_name}.')

In [ ]:
rgb_img = thumbnail_rgb
figure(figsize=(8,8), dpi=80)

plt.imshow(rgb_img)
plt.title("RGB image")
plt.xticks([])
plt.yticks([])
#plt.savefig( slide_name + 'Thumbnail.png', dpi=100)



In [ ]:
(thumb_height, thumb_width, _) = np.shape(thumbnail_rgb)

In [ ]:
hsv_img = rgb2hsv(thumbnail_rgb)
hue_img = hsv_img[:, :, 0]
value_img = hsv_img[:, :, 2]
saturation_img= hsv_img[:,:, 1]

fig, (ax0, ax1,) = plt.subplots(ncols=2, figsize=(20,10))

ax0.imshow(thumbnail_rgb)
ax0.set_title("RGB image")
ax0.axis('off')

ax1.imshow(hsv_img[:, :, 1], cmap = plt.cm.gray)
ax1.set_title("Saturation_channel")
ax1.axis('off')

plt.show()
fig.tight_layout()


In [ ]:
thresh_img = saturation_img
threshold = 0.2
binary_img = thresh_img > np.max(thresh_img)*(threshold)

selem = disk(1)
mask_proc = opening(binary_img, selem)
mask_proc = ndi.binary_fill_holes(mask_proc>0)

mask_proc = remove_small_objects(mask_proc, min_size = 1000)
labeled_proc = label(mask_proc, background = 0)

fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(20,10))

ax0.imshow(binary_img)
ax0.set_title("Thresholded image")
ax0.axis('off')

ax1.imshow(labeled_proc)
ax1.set_title("Filled image")
ax1.axis('off')

plt.show()
fig.tight_layout()

In [ ]:
selem = disk(1)
mask_proc = opening(binary_img, selem)
plt.imshow(mask_proc)
mask_proc = ndi.binary_fill_holes(mask_proc>0)

f, ax = plt.subplots(1, 4, figsize=(20, 20))
ax[0].imshow(binary_img)
ax[1].imshow(mask_proc, cmap=plt.cm.gray)

for a in ax:
    a.axis('off')
plt.show()

In [ ]:
mask_proc = remove_small_objects(mask_proc, min_size = 1000)
labeled_proc = label(mask_proc, background = 0)
plt.imshow(labeled_proc)

In [ ]:
def detect_tissue_boundary(img, threshold = 0.07):
  
    # Convert to HSV
    hsv_img = rgb2hsv(img)
    
    # Split the channels
    hue_img = hsv_img[:, :, 0]
    saturation_img= hsv_img[:,:, 1]
    value_img = hsv_img[:, :, 2]
    
    # Threshold the saturation channel
    thresh_img = saturation_img
    binary_img = thresh_img > np.max(thresh_img)*threshold

    # Run binary opening
    selem = disk(1)
    mask_proc = opening(binary_img, selem)
    
    #plt.imshow(mask_proc)
    # Fill holes
    mask_proc = ndi.binary_fill_holes(mask_proc>0)
    labeled_proc = label(mask_proc, background = 0)
    
    return labeled_proc

In [ ]:
labeled_img = detect_tissue_boundary(thumbnail_rgb, threshold = 0.06)

In [ ]:
# Plot out the image and detected boundary

f, ax = plt.subplots(1,2, figsize=(20,10))

ax[0].imshow(thumbnail_rgb)
ax[1].imshow(labeled_img)

for a in ax:
    a.axis('off')
    
plt.show()
f.tight_layout()

In [ ]:
# Find the largest object in the image
largestCC = labeled_img == np.argmax(np.bincount(labeled_img.flat)[1:]) + 1




In [ ]:
# Plot out the image and detected boundary

f, ax = plt.subplots(1,2, figsize=(20,10))

ax[0].imshow(thumbnail_rgb)
ax[1].imshow(largestCC)

for a in ax:
    a.axis('off')
    
plt.show()
f.tight_layout()

## Find tissue boundaries and scale their coordinates

The tissue boundary coordinates need to be scaled to the full image size; so here we figure out how much to multiply each coordinate by.

This will allow us to perform feature extraction / graph creation in the original WSI coordinate space.

In [ ]:
# Grab the x and y coordinates of the tissue regions
def get_tissue_coordinates(labeled_img):
    img_tissue_boundary = segmentation.find_boundaries(labeled_img)
    print(np.shape(labeled_img))
    
    (tissue_y, tissue_x) = np.nonzero(img_tissue_boundary)
    print(f'Image_boundary {np.shape(img_tissue_boundary)}')

    tissue_x = (tissue_x).astype(float)
    tissue_y = (tissue_y).astype(float)
    return tissue_x, tissue_y

In [ ]:
def get_tissue_boundaries(labeled_img):
    
    # Get the list of labels for this class
    objs = np.unique(labeled_img)
    
    obj_bounds = []

    for obj in objs[1:]:
        # Grab a binary image containing only the current object
        this_obj = labeled_img == obj
        
        # Pad by 1 -- need to operate correctly at the boundaries
        this_obj = np.pad(this_obj, 1, 'constant', constant_values=0)

        # Use the contour function to grab the boundary points IN CORRECT ORDER
        # The order of the contour points is important; Histomics will freak out
        # if the points are not in "marching" order around the boundary
        cnt = plt.contour(this_obj)
        pts = cnt.collections[0].get_paths()[0].vertices

        obj_bounds.append(pts)
    
    return obj_bounds

In [ ]:
#tissue_x, tissue_y = get_tissue_coordinates(labeled_img)
tissue_bounds = get_tissue_boundaries(largestCC)

In [ ]:
# Unravel tissue bounds to get all X's and Y's
tissue_x = []
tissue_y = []
for tissue_bound in tissue_bounds:
    for a in tissue_bound:
        tissue_x.append(a[0])
        tissue_y.append(a[1])
num_pts = len(tissue_x)


# gets every 3 points of tissue coordinates
tissue_x = np.array(tissue_x[::3])
tissue_y = np.array(tissue_y[::3])

In [ ]:
slide_info = conn.get(f"/item/{slide_id}/tiles")

In [ ]:
def get_slide_ratio(slide_info, thumb_height, thumb_width):
    slide_width = slide_info['sizeX']
    slide_height = slide_info['sizeY']

    height_ratio = slide_height / thumb_height
    width_ratio = slide_width / thumb_width
    
    return height_ratio, width_ratio

In [ ]:
height_ratio, width_ratio = get_slide_ratio(slide_info, thumb_height, thumb_width)

In [ ]:
height_ratio

In [ ]:
tissue_x = list((width_ratio * tissue_x).astype(int))
tissue_y = list((height_ratio * tissue_y).astype(int))


tissue_cordinates = [[x,y] for x,y in zip (tissue_x, tissue_y)]

In [ ]:
elements, metadata = get_annotations_from_htk(conn, slide_id, group_list=['tumor', 'ai_tumor', 'satellite'])

In [ ]:
def get_edge_coordinates(elements):
    coords = []
    for idx, element in enumerate(elements):
        points = element['points']
        points = [x[:-1] for x in points]
        
        X = np.array([int(p[0]) for p in points], dtype=np.int64)
        Y = np.array([int(p[1]) for p in points], dtype=np.int64)
        
        coords.append([X, Y])
    return coords


In [ ]:
def get_polygon_from_coords(coords):
    polygons = []
    for coord in coords:
        X = coord[0]
        Y = coord[1]
        point_list = [(x,y) for x,y in zip(X,Y)]
        poly = Polygon(point_list)
        polygons.append(poly)
    return polygons

In [ ]:
def get_polygon_objects(elements):
    # Get the edges of the polygons and merge those which are overlapping
    poly_edges = get_edge_cordinates(elements)
    polygon_objects = get_polygon_from_cords(poly_edges)
    poly_union = unary_union(polygon_objects)
    print(type(poly_union))
    
    # Calculate the centroids of all the unified polygons
    polygon_centroids = np.array([0, 0])
    if type(poly_union) == Polygon:
        print('True')
        x,y = poly_union.centroid.xy
        polygon_centroids = np.vstack([polygon_centroids, [x[0], y[0]]])
    else:

        
        for poly in poly_union:
            x,y = poly.centroid.xy
            polygon_centroids = np.vstack([polygon_centroids, [x[0], y[0]]])
    polygon_centroids = polygon_centroids[1:]
    return poly_union, polygon_centroids

In [ ]:
poly_union, polygon_centroids = get_polygon_objects(elements)

# Cut the polygons into neighborhoods

The process here:

1. Construct DT
2. Break the branches that cross tissue coordinates
3. Process each neighborhood to create a unique geometry file

In [ ]:
# Construct DT
tri = Delaunay(polygon_centroids)
#tri_simplices_cut = cut_triangles(tri, tissue_cordinates)


In [ ]:
# Display
f, ax = plt.subplots(figsize=(10,10))

ax.triplot(polygon_centroids[:,0], polygon_centroids[:,1], tri.simplices, c='r')
ax.scatter(polygon_centroids[:,0], polygon_centroids[:,1], c='g')
ax.scatter(tissue_x, tissue_y)
ax.axis('off')
plt.savefig( slide_name + 'Initial_Delaunay.png', dpi=100)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(10,10))

ax.triplot(polygon_centroids[:,0], polygon_centroids[:,1], tri.simplices, c='r')
ax.scatter(polygon_centroids[:,0], polygon_centroids[:,1], c='g')
ax.scatter(tissue_x, tissue_y)

plt.xlim(13000, 19500)
plt.ylim(5000, 12000)

#plt.savefig( slide_name + 'Initial_Delaunay.png', dpi=100)
plt.show()



In [ ]:
len(tissue_cordinates)

In [ ]:
#tissue_cordinates = sorted(tissue_cordinates)
tissue_cordinates=np.array(tissue_cordinates)

In [ ]:
def direction(p1, p3, p4):
    diff13 = np.subtract(p1, p3)
    diff43 = np.subtract(p4, p3)

    return np.cross(diff13, diff43)

In [ ]:
def on_segment(p1, p2, p):
    return min(p1[0], p2[0]) <= p[0] <= max(p1[0], p2[0]) and min(p1[1], p2[1]) <= p[1] <= max(p1[1], p2[1])

In [ ]:
def check_intersection(pt1, pt2, pt3, pt4):
    """Check whether the lines represented by pt1->pt2 and pt3->pt4 """
    d1 = direction(pt1, pt3, pt4)
    d2 = direction(pt2, pt3, pt4)

    d3 = direction(pt3, pt1, pt2)
    d4 = direction(pt4, pt1, pt2)

    if ((d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0)) and \
        ((d3 > 0 and d4 < 0) or (d3 < 0 and d4 > 0)):
        return True
    elif d1 == 0 and on_segment(pt3, pt4, pt1):
        return True
    elif d2 == 0 and on_segment(pt3, pt4, pt2):
        return True
    elif d3 == 0 and on_segment(pt1, pt2, pt3):
        return True
    elif d4 == 0 and on_segment(pt1, pt2, pt4):
        return True
    else:
        return False




In [ ]:

pt1 = np.array([0, 0])
pt2 = np.array([1, .6])

pt3 = np.array([0, 0])
pt4 = np.array([1.1, 1])

f,a = plt.subplots(figsize=(10,10))

a.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'k')
a.plot([pt3[0], pt4[0]], [pt3[1], pt4[1]], 'r')

a.text(pt1[0], pt1[1], 'Pt 1')
a.text(pt2[0], pt2[1], 'Pt 2')
a.text(pt3[0], pt3[1], 'Pt 3')
a.text(pt4[0], pt4[1], 'Pt 4')

plt.show()

check_intersection(pt1, pt2, pt3, pt4)

In [ ]:
# PART 2
# create a list of lines for each pair of tissue coordinate
tissue_lines = []
end  = len(tissue_cordinates)-1
for idx in np.arange(len(tissue_cordinates)):
    if idx == 0:
        this_line = [tissue_cordinates[end], tissue_cordinates[idx]]
    else:
        this_line = [tissue_cordinates[idx-1], tissue_cordinates[idx]]
        
    tissue_lines.append(this_line)



In [ ]:
# test tissue coordinates that you are sure have DT intersection
#test_tissue_lines = tissue_lines[1000:1800]



In [ ]:
# Display
f, ax = plt.subplots(figsize=(10,10))

ax.triplot(polygon_centroids[:,0], polygon_centroids[:,1], tri.simplices, c='b')
ax.scatter(polygon_centroids[:,0], polygon_centroids[:,1], c='g')
ax.scatter(tissue_x, tissue_y)
ax.scatter(tissue_x, tissue_y)
for test_line in tissue_lines:
    ax.plot(
        [test_line[0][0], test_line[1][0]], 
        [test_line[0][1], test_line[1][1]], 'r')
ax.axis('off')

#plt.xlim(13000, 19500)
#plt.ylim(5000, 12000)

plt.show()



In [ ]:
# perform cut triangles before performing the check intersection for WSI that contains large amounts of annotation.

#tri_simplices_cut = cut_triangles(tri.simplices, tissue_cordinates)




In [ ]:
#PART 3
# Create lines using triangle points

cut_triangles = []

for idx, t in enumerate(tri.simplices):
    p1 = polygon_centroids[t[0], :]
    p2 = polygon_centroids[t[1], :]
    p3 = polygon_centroids[t[2], :]
    
    line1 = [np.array([p1[0], p1[1]]), np.array([p2[0], p2[1]])]   
    line2 = [np.array([p2[0], p2[1]]), np.array([p3[0], p3[1]])]
    line3 = [np.array([p3[0], p3[1]]), np.array([p1[0], p1[1]])]    
    
    edges = [line1, line2, line3]

    for edge in edges:
        is_intersected = False
        #print(f'Edge: {edge}')
        
        for tissue_line in tissue_lines:
            #print(f'Tissue Line: {tissue_line}')
            
            #for p1, p2 in edge[0], edge[1]:
            #    for p3, p4 in tissue_line:
            #        print(f'Checking: p1 = {p1}, p2 = {p2}, p3 = {p3}, p4 = {p4}')
            if check_intersection(edge[0], edge[1], tissue_line[0], tissue_line[1]) == True:
                #print(f'Found an intersection!')

                is_intersected = True
                if idx not in cut_triangles:
                    cut_triangles.append(idx)
                        
            if is_intersected ==True:
                continue     

In [ ]:
len(cut_triangles)

In [ ]:
len(tri.simplices)

In [ ]:

#type(tri.simplices)

tri_simplices = (tri.simplices).tolist()


In [ ]:
# From: https://thispointer.com/python-remove-elements-from-list-by-index/

def delete_multiple_element(list_object, indices):
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx)

In [ ]:
cut_tri = cut_triangles  # idx of the triangles that need to be eliminated from triagnles

triangles = tri_simplices

delete_multiple_element(triangles, cut_tri)
    




In [ ]:
# Check the cutting
f, ax = plt.subplots(figsize=(10,10))

ax.triplot(polygon_centroids[:,0], polygon_centroids[:,1], triangles, c='r')
ax.scatter(polygon_centroids[:,0], polygon_centroids[:,1], c='g')
ax.scatter(tissue_x, tissue_y)
ax.axis('off')
plt.savefig( slide_name + 'Cut_Delaunay.png', dpi=100)
plt.show()

In [ ]:
# Display
f, ax = plt.subplots(figsize=(10,10))

ax.triplot(polygon_centroids[:,0], polygon_centroids[:,1], triangles, c='b')
ax.scatter(polygon_centroids[:,0], polygon_centroids[:,1], c='g')
ax.scatter(tissue_x, tissue_y)
ax.scatter(tissue_x, tissue_y)
for test_line in tissue_lines:
    ax.plot(
        [test_line[0][0], test_line[1][0]], 
        [test_line[0][1], test_line[1][1]], 'r')
ax.axis('off')

#plt.xlim(13000, 19500)
#plt.ylim(5000, 12000)

plt.show()

In [ ]:



f, ax = plt.subplots(figsize=(15,15))

ax.triplot(polygon_centroids[:,0], polygon_centroids[:,1], triangles, c='b')























In [ ]:
dt_lengths = get_triangle_lengths(polygon_centroids, triangles)
dt_areas = get_triangle_areas(polygon_centroids, triangles)
dt_lengths_feats = descriptive_stats(dt_lengths, 'delaunay_length_')
dt_areas_feats = descriptive_stats(dt_areas, 'delaunay_area_')

In [ ]:
total_dt_areas = sum(dt_areas)

In [ ]:
total_dt_areas

In [ ]:
# Upload features to SUNY CELL WSI Folder



def create_delaunay_dataframe(nodes, simplices, group_name='Delaunay Triangulation', color_string='rgb(0,0,0)'):
    group = []
    color = []
    ymin = []
    ymax = []
    xmin = []
    xmax = []
    has_holes = []
    touches_edge_top = []
    touches_edge_left = []
    touches_edge_bottom = []
    touches_edge_right = []
    coords_x = []
    coords_y = []

    for simplex in simplices:
        # Grab the coordinates of the triangle
        obj_x = nodes[simplex, 0]
        obj_y = nodes[simplex, 1]

        # =========
        # Possibly need to check that these should be str(int(x)) and str(int(y))?
        obj_x_str = ','.join([str(int(x)) for x in obj_x])
        obj_y_str = ','.join([str(int(y)) for y in obj_y])
        # =========

        ymin.append(int(np.min(obj_y)))
        xmin.append(int(np.min(obj_x)))
        ymax.append(int(np.max(obj_y)))
        xmax.append(int(np.max(obj_x)))

        coords_x.append(obj_x_str)
        coords_y.append(obj_y_str)

        # Properties of the overall class group (Delaunay)
        group.append(group_name)
        color.append(color_string)

        has_holes.append(0)
        touches_edge_top.append(0)
        touches_edge_left.append(0)
        touches_edge_bottom.append(0)
        touches_edge_right.append(0)

    # Put it all into a dataframe
    return pd.DataFrame({
        'group': group,
        'color': color,
        'ymin': ymin,
        'ymax': ymax,
        'xmin': xmin,
        'xmax': xmax,
        'has_holes': has_holes,
        'touches_edge-top': touches_edge_top,
        'touches_edge-left': touches_edge_left,
        'touches_edge-bottom': touches_edge_bottom,
        'touches_edge-right': touches_edge_right,
        'coords_x': coords_x,
        'coords_y': coords_y
    })

In [ ]:
delaunay_df = create_delaunay_dataframe(polygon_centroids, triangles)

In [ ]:
        
annprops = {
'X_OFFSET': 0,
'Y_OFFSET': 0,
'opacity': 0.2,
'lineWidth': 4.0,
}
annotation_docs = get_annotation_documents_from_contours(
delaunay_df.copy(),
separate_docs_by_group=True,
annots_per_doc=200,
docnamePrefix='Delaunay_triangulation',
annprops=annprops,
verbose=False,
monitorPrefix=slide_names[0] + ": annotation docs")


ad = annotation_docs.copy()

for annotation_doc in ad:
    elements = annotation_doc['elements']
    for element in elements:
        element['closed'] = False
        element['points'] = element['points'][:-1]

# Post the annotation documents you created to the server
for annotation_doc in annotation_docs:
    resp = conn.post(
        "/annotation?itemId=" + slide_ids[18], json=annotation_doc)

In [ ]:
#Create a network graph
G = nx.Graph()

# Add points and edges to graph
#for simplex in tri_simplices_cut:
for simplex in tri.simplices:
    
    G.add_nodes_from(simplex)
    G.add_edge(simplex[0], simplex[1])
    G.add_edge(simplex[1], simplex[2])
    G.add_edge(simplex[2], simplex[0])

In [ ]:

# Display some stuff about the network
print(f'Network has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges')

nx.draw(G, node_size=1)
plt.show()

In [ ]:
# Find the separated components in this list
components = [c for c in nx.connected_components(G)]
print(f'Found {len(components)} connected components')

In [ ]:
# Plot each component on the image
f, ax = plt.subplots(figsize=(10,10))
for component_idx, component in enumerate(components):
    this_component_list = list(component)
    this_component_coords = polygon_centroids[this_component_list,:]
    this_component_tri = Delaunay(this_component_coords)
    this_component_tri_cut = cut_triangles(this_component_tri, tissue_cordinates, verbose=False)

    ax.triplot(this_component_coords[:,0], this_component_coords[:,1], this_component_tri_cut)
    ax.scatter(tissue_x, tissue_y, c='k')
    ax.axis('off')
plt.savefig( slide_name + 'Components.png', dpi=100)
plt.show()

In [ ]:
component_tissue_polys = []

for component in components:
    this_component_list = list(component)

    # We can't index directly into a multipolygon, so pull them out one at a time
    this_component_polys = []
    for this_component in this_component_list:
        this_component_polys.append(poly_union[this_component])
    component_tissue_polys.append(this_component_polys)

In [ ]:
# Apply heuristic to this set of polygons
def apply_heuristic(tissue_polys):
    poly_areas = []
    for p in tissue_polys:
        poly_areas.append(p.area)
    biggest_polygon = np.argmax(poly_areas)
    
    # Gather up all the coordinates we need
    mt_poly = tissue_polys[biggest_polygon]
    sat_polys = tissue_polys
    del sat_polys[biggest_polygon]
    
    return mt_poly, sat_polys

In [ ]:
len(component_tissue_polys)

In [ ]:
mt_poly, sat_polys = apply_heuristic(component_tissue_polys[0])

In [ ]:
# Display the largest polygon 
f, ax = plt.subplots(figsize=(10,10))
ax.imshow(thumbnail_rgb)

# Display the mt poly
x, y = mt_poly.exterior.xy
ax.plot(np.array(x) / width_ratio, np.array(y) / height_ratio, 'r') 

# Display the sat polys
for sat_poly in sat_polys:
    x, y = sat_poly.exterior.xy
    ax.plot(np.array(x) / width_ratio, np.array(y) / height_ratio, 'b') 

ax.axis('off')
plt.savefig(slide_name + 'Heuristic_WSI.png', dpi=100)
plt.show()

In [ ]:
def get_poly_bounds_and_centroid(input_poly):
    boundaries_x, boundaries_y = input_poly.exterior.xy
    boundaries = np.array([[x,y] for x,y in zip(boundaries_x, boundaries_y)])

    cent_x,cent_y = input_poly.centroid.xy
    centroid = np.array([[x,y] for x,y in zip(cent_x, cent_y)])
    return boundaries, centroid

In [ ]:
# mt_boundaries_x, mt_boundaries_y = mt_poly.exterior.xy
# mt_boundaries = np.array([[x,y] for x,y in zip(mt_boundaries_x, mt_boundaries_y)])

# tum_x,tum_y = mt_poly.centroid.xy
# mt_centroid = np.array([[x,y] for x,y in zip(tum_x, tum_y)])
mt_boundaries, mt_centroid = get_poly_bounds_and_centroid(mt_poly)


In [ ]:
# Process the satellite coordinates to be numpy arrays
sat_boundaries = []
sat_centroids = np.array([0, 0])

for sat_poly in sat_polys:

    this_sat_boundaries, this_sat_centroid = get_poly_bounds_and_centroid(sat_poly)
    sat_centroids = np.vstack([sat_centroids, this_sat_centroid])
    sat_boundaries.append(this_sat_boundaries)
    
#     x_centroid,y_centroid = sat_poly.centroid.xy
#     sat_centroids = np.vstack([sat_centroids, [[x,y] for x,y in zip(x_centroid, y_centroid)]])

#     sat_boundaries_x, sat_boundaries_y = sat_poly.exterior.xy
#     sat_boundaries.append(np.array([[x,y] for x,y in zip(sat_boundaries_x, sat_boundaries_y)]))

sat_centroids = sat_centroids[1:]

In [ ]:
cut_coordinates = np.vstack([tissue_cordinates, mt_boundaries])


tri = Delaunay(sat_centroids)
tri_simplices_cut = cut_triangles(tri, cut_coordinates)


#Check the cutting
f, ax = plt.subplots(figsize=(10,10))

ax.triplot(sat_centroids[:,0], sat_centroids[:,1], tri_simplices_cut, c='r')
ax.scatter(sat_centroids[:,0], sat_centroids[:,1], c='g')
ax.scatter(tissue_x, tissue_y, c='k')
ax.scatter(mt_boundaries[:,0], mt_boundaries[:,1], c='b')
plt.savefig(slide_name  + 'Final_Delaunay.png', dpi=100)
ax.axis('off')
plt.show()

In [ ]:
3

In [ ]:
Step1) We will loop through each of the slide id and the slide name(as shown below in the for loop)
step2) Then we go and process the WSI to get the Slide Thumbnail and detect the tissue boundary
Step3) Once we have the tissue boundary, we extract the tissue cordinates
Step4) We use these Tissue cordinates to constrcut tissue lines(Your function call it here in the for loop)
Step5) Then you go in and check the intersection of the DT with the Tissue lines
Step6) If true, delete the lines or the triangles that intersect.
Step7) Save the results

## Looping through each and every single slide to extract geometry and features.

In [ ]:
# Loop through the list of slides
for (slide_id, slide_name) in zip(slide_ids, slide_names):
    print(f'Processing {slide_name} ({slide_id})')
    
    # Skip this slide if we already have features extracted
    feature_list = glob.glob(os.path.join(feature_dir, slide_name.split('.')[0] + f'*.csv'))
    if len(feature_list)>0:
        print(f'We already have features for {slide_name}.')
        continue
        
    
    
    # grab a thumbnail of the image
    try:
        thumbnail_rgb = get_slide_thumbnail(conn, slide_id)
        (thumb_height, thumb_width, _) = np.shape(thumbnail_rgb)
    except:
        print(f'Could not extract thumbnail for {slide_name}.')
        continue
    
    # Get the connected component labels of the tissue
    labeled_proc = detect_tissue_boundary(thumbnail_rgb, threshold = 0.2)

    # Grab the x and y coordinates of the tissue regions
    tissue_x, tissue_y = get_tissue_coordinates(labeled_proc)
    
    # Grab the sample height and width, and use these to calculate the resize ratio
    try:
        slide_info = conn.get(f"/item/{slide_id}/tiles")
    except:
        print(f'Error getting slide info for {slide_id}')
        continue
    
    # Scale the tissue x and y coordinates
    height_ratio, width_ratio = get_slide_ratio(slide_info, thumb_height, thumb_width)
    
    tissue_x = list((width_ratio * tissue_x).astype(int))
    tissue_y = list((height_ratio * tissue_y).astype(int))
    
    tissue_cordinates = [[x,y] for x,y in zip (tissue_x, tissue_y)]

    
    #This is the point where you need  to add your stuff    #f, ax = plt.subplots(figsize=(5,5))
    #ax.scatter(tissue_x, tissue_y)
    #ax.axis('off')
    #plt.show()
    
    # Get the elements (slide annotations) from the server
    try:
        elements, metadata = get_annotations_from_htk(conn, slide_id, group_list=['tumor', 'ai_tumor', 'satellite'])
    except:
        print(f'Error getting annotations from {slide_name}.')
        continue
    
    # Get the edges of the polygons and merge those which are overlapping
    poly_union, polygon_centroids = get_polygon_objects(elements)
    if np.shape(polygon_centroids)[0] <= 3:
        print(f'Not enough polygon centroids for {slide_name}. Skipping.')
        continue

    print("Shape of the array = ",np.shape(polygon_centroids))
    
    # Construct DT
    tri = Delaunay(polygon_centroids)
        
    # Cut the branches using tissue boundary coordiantes
    tri_simplices_cut = cut_triangles(tri, tissue_cordinates)
    
#     # Check the cutting
#     f, ax = plt.subplots(figsize=(10,10))

#     ax.triplot(polygon_centroids[:,0], polygon_centroids[:,1], tri_simplices_cut, c='r')
#     ax.scatter(polygon_centroids[:,0], polygon_centroids[:,1], c='g')
#     ax.scatter(tissue_x, tissue_y)
#     ax.axis('off')
#     plt.savefig( name[i] + 'Cut_poly.png', dpi=100)
#     plt.show()

    #Create a network graph
    G = nx.Graph()

    # Add points and edges to graph
    for simplex in tri_simplices_cut:
        G.add_nodes_from(simplex)
        G.add_edge(simplex[0], simplex[1])
        G.add_edge(simplex[1], simplex[2])
        G.add_edge(simplex[2], simplex[0])
        
    # Display some stuff about the network
    print(f'Network has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges')
    
#     nx.draw(G, node_size=1)
#     plt.savefig(name[i] + 'Networkx_graph.png', dpi=100)
#     plt.show()
    
    # Find the separated components in this list
    components = [c for c in nx.connected_components(G)]
    print(f'Found {len(components)} connected components')
    
#     f, ax = plt.subplots(figsize=(10,10))

#     for component_idx, component in enumerate(components):
#         this_component_list = list(component)
#         this_component_coords = polygon_centroids[this_component_list,:]
#         this_component_tri = Delaunay(this_component_coords)
#         this_component_tri_cut = cut_triangles(this_component_tri, tissue_cordinates, verbose=False)

#         ax.triplot(this_component_coords[:,0], this_component_coords[:,1], this_component_tri_cut)

#         ax.scatter(tissue_x, tissue_y, c='k')
#         ax.axis('off')
#         plt.savefig( name[i] + component_idx + 'Separated_components.png', dpi=100)
#         plt.show()
    
    #type(poly_union)

    component_tissue_polys = []
    max_poly_count = 0
    for component in components:
        this_component_list = list(component)

        # We can't index directly into a multipolygon, so pull them out one at a time
        this_component_polys = []
        
        for this_component in this_component_list:
            this_component_polys.append(poly_union[this_component])
        component_tissue_polys.append(this_component_polys)
        if max_poly_count < len(this_component_polys):
            max_poly_count = len(this_component_polys)
            
        
        
    for tissue_idx, tissue_polys in enumerate(component_tissue_polys):
        if len(tissue_polys) < max_poly_count:
            continue
        save_name = slide_name + f'_{tissue_idx}.csv'
        
        mt_poly, sat_polys = apply_heuristic(tissue_polys)
                
        mt_boundaries, mt_centroid = get_poly_bounds_and_centroid(mt_poly)

        # Get satellite boundaries and centroids
        sat_boundaries = []
        sat_centroids = np.array([0, 0])

        for sat_poly in sat_polys:
            this_sat_boundaries, this_sat_centroid = get_poly_bounds_and_centroid(sat_poly)
            sat_centroids = np.vstack([sat_centroids, this_sat_centroid])
            sat_boundaries.append(this_sat_boundaries)
        sat_centroids = sat_centroids[1:]
        
        # Cut the satellites again
        cut_coordinates = np.vstack([tissue_cordinates, mt_boundaries])

        if len(sat_centroids) < 3:
            print(f'{slide_name}, in component {tissue_idx}, has insufficient satellites (only has {len(sat_centroids)}, so skipping')
            continue
            
        tri = Delaunay(sat_centroids)
        tri_simplices_cut = cut_triangles(tri, cut_coordinates)

                
        # Check the cutting
#         f, ax = plt.subplots(figsize=(10,10))

#         ax.triplot(sat_centroids[:,0], sat_centroids[:,1], tri_simplices_cut, c='r')
#         ax.scatter(sat_centroids[:,0], sat_centroids[:,1], c='g')
#         ax.scatter(tissue_x, tissue_y, c='k')
#         ax.scatter(mt_boundaries[:,0], mt_boundaries[:,1], c='b')
#         #plt.savefig(slide_name  + 'Final_Delaunay.png', dpi=100)
#         ax.axis('off')
#         plt.show()
        try:
            dt_lengths = get_triangle_lengths(sat_centroids, tri_simplices_cut)
            dt_areas = get_triangle_areas(sat_centroids, tri_simplices_cut)
            dt_lengths_feats = descriptive_stats(dt_lengths, 'delaunay_length_')
            dt_areas_feats = descriptive_stats(dt_areas, 'delaunay_area_')
        except:
            print(f'{slide_name} has insufficient unique, non-negative simplex cordinates in {tissue_idx}, so skipping')

            pass

#         # We already have our DT from above
#         dt_lengths = get_triangle_lengths(sat_centroids, tri_simplices_cut)
#         dt_areas = get_triangle_areas(sat_centroids, tri_simplices_cut)
#         dt_lengths_feats = descriptive_stats(dt_lengths, 'delaunay_length_')
#         dt_areas_feats = descriptive_stats(dt_areas, 'delaunay_area_')
        
        sat_surf_distances = []
        for sat_bound in sat_boundaries:
            sat_surf_distances.append(distance.cdist(sat_bound, mt_boundaries, 'euclidean').min())

        sat_surf_distances_features = descriptive_stats(sat_surf_distances, 'sat_surf_distance_')

        # Calculate min distances directly from the sat / mt coordinates
        sat_centroid_distances = np.amin(distance.cdist(sat_centroids, mt_boundaries, 'euclidean'), axis=1)

        #upload distance df
        #distance_df = create_distance_dataframe(sat_surf_distances, sat_centroids)

        sat_centroid_distances_features = descriptive_stats(sat_centroid_distances, 'sat_absolute_distance_')

        # Compute convex hull for satellite boundaries
        sat_boundaries_stacked = np.vstack(sat_boundaries)
        sat_hull = ConvexHull(sat_boundaries_stacked)
        mt_hull = ConvexHull(mt_boundaries)


        sat_pct_area = []
        for sat_poly in sat_polys:
            sat_pct_area.append(sat_poly.area / (sat_hull.volume - mt_hull.volume))
        sat_hull_features = descriptive_stats(sat_pct_area, 'sat_hull_pct_area_')

        # Convert coordinates to thumbnail-size
        mt_boundaries_y = (mt_boundaries[:,0] / height_ratio).astype(int)
        mt_boundaries_x = (mt_boundaries[:,1] / width_ratio).astype(int)

        # Create the mt binary mask
        mt_mask = Image.new('L', (thumb_width, thumb_height), 0)
        ImageDraw.Draw(mt_mask).polygon([(x,y) for x,y in zip(mt_boundaries_y, mt_boundaries_x)], outline=1, fill=1)
        mt_mask = np.array(mt_mask)

        # Convert sat coordinates to thumbnail-size
        sat_scaled = []
        for sat_boundary in sat_boundaries:
            sat_scaled.append(sat_boundary / [width_ratio, height_ratio])

        f, ax = plt.subplots(figsize=(10,10))
        ax.imshow(mt_mask)
        for sat in sat_scaled:
            ax.scatter(sat[:,0], sat[:,1])
        ax.axis('off')
        #plt.savefig(slide_name  + 'Convex_Hull.png', dpi=100)
        plt.show()

        # Creating satellite wave numbers
        sat_wave_number = assign_wave_index(mt_mask, sat_scaled)

        #Creating graphs
        sat_wave_indices = np.reshape(sat_wave_number, [len(sat_wave_number),])

        # Get the list of descending-size arrays
        sat_ordering = np.argsort(sat_wave_indices)[::-1]

        wave_distances = []

        # For each satellite
        for sat_idx, sat in enumerate(sat_polys):

            # Get the sat boundaries 
            curr_sat_boundaries = sat_boundaries[sat_idx]

            # Get a list of the satellites + mt that have a lower wave index
            wave_targets = np.where(sat_wave_indices < sat_wave_indices[sat_idx])
            sat_targets = []
            for wave_target in wave_targets[0]:
                sat_targets.append(sat_polys[wave_target])
            sat_targets.append(mt_poly)

            # Calculate the distance between this current sat and all targets
            sat_target_distances = []
            for sat_target in sat_targets:
                # Calculate the minimum distance from the surface of the current sat to the surface of the target
                target_x, target_y = sat_target.exterior.xy
                target_boundaries = np.array([[x,y] for x,y in zip(target_x, target_y)])
                sat_target_distances.append(distance.cdist(target_boundaries, curr_sat_boundaries, metric = 'euclidean').min())

            # Figure out which target is the closest
            closest_target_distance = np.min(np.array(sat_target_distances))
            wave_distances.append(closest_target_distance)

        wave_features = descriptive_stats(wave_distances, 'wave_distance_')
        
        features_dataframe = pd.concat([dt_lengths_feats, dt_areas_feats, sat_surf_distances_features, sat_centroid_distances_features, sat_hull_features, wave_features], axis=1)

        #save_name = slide_name + f'_{tissue_idx}.csv'

        # Set the index of the feature dataframe to be the save-name
        features_dataframe['slide_tissue'] = save_name.split('.')[0]
        features_dataframe = features_dataframe.set_index('slide_tissue')
        #features_dataframe.to_csv(save_dir / save_name)
        features_dataframe.to_csv(save_name)

In [ ]:
#Extra Stuff that can be used to visualize the features. 

## Script to upload Features on Histomics
The functions below take the different feature values that have been extracted and uploads them on Histomics as a different group which can be visualized.

In [ ]:
def create_distance_dataframe(pts1, pts2, group_name='Satellite Distances', color_string='rgb(1,0,0)'):
    """Takes in two point sets (pts1, pts2) and returns a DSA-style dataframe consisting of the lines between the two.
    """
    group = []
    color = []
    ymin = []
    ymax = []
    xmin = []
    xmax = []
    has_holes = []
    touches_edge_top = []
    touches_edge_left = []
    touches_edge_bottom = []
    touches_edge_right = []
    coords_x = []
    coords_y = []
    
    assert len(pts1) == len(pts2)
    npts = len(pts1)
    
    for pt_idx in np.arange(npts):
        
        obj_x = [pts1[pt_idx][0], pts2[pt_idx][0]]
        obj_y = [pts1[pt_idx][1], pts2[pt_idx][1]]

        # =========
        # Possibly need to check that these should be str(int(x)) and str(int(y))?
        obj_x_str = ','.join([str(int(x)) for x in obj_x])
        obj_y_str = ','.join([str(int(y)) for y in obj_y])
        # =========

        ymin.append(int(np.min(obj_y)))
        xmin.append(int(np.min(obj_x)))
        ymax.append(int(np.max(obj_y)))
        xmax.append(int(np.max(obj_x)))

        coords_x.append(obj_x_str)
        coords_y.append(obj_y_str)

        # Properties of the overall class group (Delaunay)
        group.append(group_name)
        color.append(color_string)

        has_holes.append(0)
        touches_edge_top.append(0)
        touches_edge_left.append(0)
        touches_edge_bottom.append(0)
        touches_edge_right.append(0)

    # Put it all into a dataframe
    return pd.DataFrame({
        'group': group,
        'color': color,
        'ymin': ymin,
        'ymax': ymax,
        'xmin': xmin,
        'xmax': xmax,
        'has_holes': has_holes,
        'touches_edge-top': touches_edge_top,
        'touches_edge-left': touches_edge_left,
        'touches_edge-bottom': touches_edge_bottom,
        'touches_edge-right': touches_edge_right,
        'coords_x': coords_x,
        'coords_y': coords_y
    })

In [ ]:
def create_delaunay_dataframe(nodes, simplices, group_name='Delaunay Triangulation', color_string='rgb(0,0,0)'):
    group = []
    color = []
    ymin = []
    ymax = []
    xmin = []
    xmax = []
    has_holes = []
    touches_edge_top = []
    touches_edge_left = []
    touches_edge_bottom = []
    touches_edge_right = []
    coords_x = []
    coords_y = []

    for simplex in simplices:
        # Grab the coordinates of the triangle
        obj_x = nodes[simplex, 0]
        obj_y = nodes[simplex, 1]

        # =========
        # Possibly need to check that these should be str(int(x)) and str(int(y))?
        obj_x_str = ','.join([str(int(x)) for x in obj_x])
        obj_y_str = ','.join([str(int(y)) for y in obj_y])
        # =========

        ymin.append(int(np.min(obj_y)))
        xmin.append(int(np.min(obj_x)))
        ymax.append(int(np.max(obj_y)))
        xmax.append(int(np.max(obj_x)))

        coords_x.append(obj_x_str)
        coords_y.append(obj_y_str)

        # Properties of the overall class group (Delaunay)
        group.append(group_name)
        color.append(color_string)

        has_holes.append(0)
        touches_edge_top.append(0)
        touches_edge_left.append(0)
        touches_edge_bottom.append(0)
        touches_edge_right.append(0)

    # Put it all into a dataframe
    return pd.DataFrame({
        'group': group,
        'color': color,
        'ymin': ymin,
        'ymax': ymax,
        'xmin': xmin,
        'xmax': xmax,
        'has_holes': has_holes,
        'touches_edge-top': touches_edge_top,
        'touches_edge-left': touches_edge_left,
        'touches_edge-bottom': touches_edge_bottom,
        'touches_edge-right': touches_edge_right,
        'coords_x': coords_x,
        'coords_y': coords_y
    })